# Neighbourhoods of Toronto - Part 3

## Import from previous part

In [2]:
import pandas as pd
import numpy as np

## Extracting the table from WikiPedia

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
dfr = pd.read_html(url)[0]
df = pd.DataFrame(dfr)
df.head()

## Removing rows with unassigned Boroughs

df1 = df[df['Borough']!='Not assigned'].reset_index(drop = True)
df1

## Assigning Borough name to unaasigned Neighbourhoods

df1.loc[df1['Neighbourhood']== 'Not assigned','Neighbourhood'] = df1.loc[df1['Neighbourhood']== 'Not assigned','Borough']
df1.head(10)

## Grouping Neighbourhoods with same Postcodes

df2 = df1.groupby(['Postcode','Borough'])['Neighbourhood'].apply(lambda x: ','.join(x.astype(str))).reset_index()
df2.head(10)

## Shape of the dataframe

df2.shape

df2.head(20)

## Import Location Data

url2 = 'https://cocl.us/Geospatial_data'
df_loc = pd.read_csv(url2)
df_loc.head()

## Joining Location Dataframe

df_joint = df2.join(df_loc.set_index('Postal Code'), on='Postcode')
df_joint.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


## Obtain coordinates of Toronto

In [3]:
from geopy.geocoders import Nominatim
address = 'Toronto, Ontario'
geolocator = Nominatim(user_agent="toronto")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [4]:
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: done

# All requested packages already installed.



In [5]:
dff = df_joint

## Map of Toronto with Neighbourhoods marked

In [8]:
import folium
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(dff['Latitude'], dff['Longitude'], dff['Borough'], dff['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Get Nearby Venues using Foursquare API

In [ ]:
CLIENT_ID = 'OREU0ESXHHUMVDEFEGAJAEVFTCQRUR2EZJWZYLBUM1A5YCUY' # your Foursquare ID
CLIENT_SECRET = '1PDNS3NFJ4UCMMFUXHADFBEST54XSFWZWT4P1QJSJ55XPRER' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 105
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

In [ ]:
import requests
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
toronto_nearby = getNearbyVenues(names=dff['Neighbourhood'],
                                   latitudes=dff['Latitude'],
                                   longitudes=dff['Longitude']
                                  )

In [ ]:
print(toronto_nearby.shape)
toronto_nearby.head()

### Neighbourhoods with count of unique category venues

In [ ]:
print('There are {} uniques categories.'.format(len(toronto_nearby['Venue Category'].unique())))
toronto_nearby.groupby('Neighborhood')['Venue Category'].nunique().to_frame().sort_values('Venue Category',ascending = False)

### Changing Venue Categories name 'Neighborhood' to 'Neighborhood Category'

In [ ]:
tn = toronto_nearby
tn.loc[tn['Venue Category']=='Neighborhood', 'Venue Category'] = 'Neighborhood Category'
tn['Venue Category']

In [ ]:
# one hot encoding
toronto_onehot1 = pd.get_dummies(tn[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot1['Neighborhood'] = tn['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot1.columns[-1]] + list(toronto_onehot1.columns[:-1])
toronto_onehot1 = toronto_onehot1[fixed_columns]
toronto_onehot1.head()


### Grouping by neighbourhood and taking mean of occurance of each category

In [ ]:
toronto_grouped = toronto_onehot1.groupby('Neighborhood').mean().reset_index()
toronto_grouped

### Neighbourhoods along with top 5 common venues

In [ ]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

### Copying it to a dataframe

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

### Clustering

In [ ]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:20] 
print(kmeans.labels_.shape)

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = dff

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood').dropna()

toronto_merged.head() # check the last columns!

In [ ]:
toronto = toronto_merged
toronto['Cluster Labels'] = toronto_merged['Cluster Labels'].astype('int32')
toronto.dtypes

## Visualize the cluster

In [ ]:
import matplotlib.cm as cm
import matplotlib.colors as colors
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto['Latitude'], toronto['Longitude'], toronto['Neighbourhood'], toronto['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine the Clusters

### Cluster 1

In [ ]:
toronto_merged.loc[toronto['Cluster Labels'] == 0, toronto.columns[[1] + list(range(5, toronto.shape[1]))]].reset_index(drop = True)

### Cluster 2

In [ ]:
toronto_merged.loc[toronto['Cluster Labels'] == 1, toronto.columns[[1] + list(range(5, toronto.shape[1]))]].reset_index(drop = True)

### Cluster 3

In [ ]:
toronto_merged.loc[toronto['Cluster Labels'] == 2, toronto.columns[[1] + list(range(5, toronto.shape[1]))]].reset_index(drop = True)

### Cluster 4

In [ ]:
toronto_merged.loc[toronto['Cluster Labels'] == 3, toronto.columns[[1] + list(range(5, toronto.shape[1]))]].reset_index(drop = True)

### Cluster 5

In [ ]:
toronto_merged.loc[toronto['Cluster Labels'] == 4, toronto.columns[[1] + list(range(5, toronto.shape[1]))]].reset_index(drop = True)